In [1]:
import sys
sys.path.append("utils/")

from Verification_Data_func import *

In [ ]:
# entropy_df_generation function
def entropy_df_generation(dic_result,n_round = 3):
    df = pd.DataFrame(columns=["token", 'logprob'])
    for i in range(n_round):
        result_i = dic_result[i]["choices"]
        if '{' in  result_i[0]["logprobs"]["tokens"]:
            index = result_i[0]["logprobs"]["tokens"].index('{')
        else:
            index = result_i[0]["logprobs"]["tokens"].index(' {')
        sample_token = result_i[0]["logprobs"]["tokens"][index+1]
        sample_token_logprob = result_i[0]["logprobs"]["token_logprobs"][index+1]
        df.loc[len(df)] = [sample_token, sample_token_logprob]  
        log_list = result_i[0]["logprobs"]["top_logprobs"][index+1]
        for key in list(log_list.keys()):
            df.loc[len(df)] = [key, log_list[key]]
    df_result = pd.DataFrame(columns=["token", 'logprob'])
    for token in np.unique(df["token"]):
        df_result.loc[len(df_result)] = [token, np.mean(df[df["token"] == token]["logprob"])]
    df_result["prob"] = np.exp(df_result["logprob"])
    df_result["prob"] = df_result["prob"]/np.sum(df_result["prob"])
    df_result["logprob"] = np.log(df_result["prob"])
    df_result = df_result.sort_values(by = "prob", ascending = False).reset_index(drop=True)
    df_result["cumprob"] = np.cumsum(df_result["prob"])
    df_result_majority = df_result[df_result["cumprob"] < 1] 
    df_result_majority = df_result_majority[[i.replace(' ','').isnumeric() for i in df_result_majority["token"]]].reset_index(drop=True)
    df_result_majority["prob"] = np.exp(df_result_majority["logprob"])
    df_result_majority["prob"] = df_result_majority["prob"]/np.sum(df_result_majority["prob"])
    df_result_majority["cumprob"] = np.cumsum(df_result_majority["prob"])
    return df_result_majority

# demo cot 4-shot ICL GSM8K testset Q10 
def Davinci_openai_stop(prompt, stop_index):
    response = openai.Completion.create(
    model="text-davinci-002",prompt = prompt, temperature = 1,
    max_tokens=1024, top_p=1, frequency_penalty=0,
    presence_penalty=0,logprobs = 5,
    stop = stop_index)
    return response

def Davinci_openai(prompt):
    response = openai.Completion.create(
    model="text-davinci-002",prompt = prompt, temperature = 1,
    max_tokens=32, top_p=1, frequency_penalty=0,
    presence_penalty=0,logprobs = 5)
    return response

def answer_diversity(ans_prompt,n_round = 3):
    count = 0 
    dic_result = {}
    while count < n_round:
        index_front  = -100
        index_end = 100 
        ans_generation = Davinci_openai(ans_prompt)
        if ("{" in ans_generation["choices"][0]["logprobs"]["tokens"]):
            index_front =  ans_generation["choices"][0]["logprobs"]["tokens"].index("{")
        if (" {" in ans_generation["choices"][0]["logprobs"]["tokens"]):
            index_front =  ans_generation["choices"][0]["logprobs"]["tokens"].index(" {")

        if ("}" in ans_generation["choices"][0]["logprobs"]["tokens"]):
            index_end =  ans_generation["choices"][0]["logprobs"]["tokens"].index("}")
        if ("} " in ans_generation["choices"][0]["logprobs"]["tokens"]):
            index_end =  ans_generation["choices"][0]["logprobs"]["tokens"].index("} ")
        
        if index_end - index_front == 2: 
            dic_result[count] = ans_generation
            count += 1 
    return dic_result

In [ ]:
def entropy_check(i,step_reasoning_list, n_shot_num = 4, n_step_num = 3):

    # answer entropy check
    ans_prompt = n_shot_prompt_generator_GSM8K_reason(n_shot_num ,n_step_num ,GSM8K_test_df["question"][i], GSM8K_train_df_reason, False, len(step_reasoning_list), True) 

    for _ in step_reasoning_list:
        ans_prompt += '\n'
        ans_prompt += _["choices"][0]["text"]
    ans_prompt += "\nThe answer is"

    ## sample 
    dic_result = answer_diversity(ans_prompt)

    ## dataprocessing
    df_summary = entropy_df_generation(dic_result)

    ## entrpy check
    entropy_value = entropy(df_summary["prob"])
    print(entropy_value)
    return ans_prompt, dic_result, df_summary, entropy_value

from functools import reduce  # forward compatibility for Python 3
import operator

def getFromDict(dataDict, mapList):
    return reduce(operator.getitem, mapList, dataDict)

def setInDict(dataDict, mapList, value):
    getFromDict(dataDict, mapList[:-1])[mapList[-1]] = value

def belif_change_index(dist1, dist2):
    index = 0
    token_list = list(set(dist1["token"]) | set(dist2["token"]))
    for i in token_list:
        if (i in list(dist1["token"])) and (i in list(dist2["token"])):
            index1 = list(dist1["token"]).index(i)
            index2 = list(dist2["token"]).index(i)
            index += np.abs(dist1["prob"][index1] - dist2["prob"][index2])

        elif (i not in list(dist1["token"])) and (i in list(dist2["token"])):
            index2 = list(dist2["token"]).index(i)
            index += np.abs(dist2["prob"][index2])        
        else:
            index1 = list(dist1["token"]).index(i)
            index += np.abs(dist1["prob"][index1])     
    return(index)

In [ ]:
def iteration(tmp_book,entropy_level = 1):
    layer_number = int(tmp_book["layer"].split("{")[1][0])
    step_str = "Step{"+str(layer_number+1) + "}Num"
    tmp = {}  
    tmp[step_str] = 0
    entropy_score = 10 
    # intial prompt 
    count = 0 
    print("In Step " + str(layer_number) + "\n")

    entropy_value = tmp_book["Status"][-1]
    dist1 = tmp_book["Status"][2]
    prompt = tmp_book["prompt"]
    step_reasoning = tmp_book["step_reasoning"]
    prompt += step_reasoning['choices'][0]["text"]

    while (entropy_score > entropy_level) &  (count < 2):
        step_reasoning_list = tmp_book["step_reasoning_list"].copy()
        print("-----------------------------------------------------") 
        step_reasoning = Davinci_openai_stop(prompt,["Step "+str(layer_number+2)])
        step_reasoning_list.append(step_reasoning)
        print('-------------------------------------------')
        for i in step_reasoning_list:
            print(i['choices'][0]["text"])
        print('-------------------------------------------')
        print(step_reasoning['choices'][0]["text"])   
        ans1 = entropy_check(question_index,step_reasoning_list , n_shot, n_step)
        dist2 = ans1[2]
        tmp[count] = {}
        tmp[count]["prompt"] = prompt
        tmp[count]["Status"] = ans1
        tmp[count]["step_reasoning"] = step_reasoning 
        tmp[count]["step_reasoning_list"] = step_reasoning_list
        tmp[step_str] += 1
        tmp[count]["layer"] = step_str
        tmp[count]["belif"] = belif_change_index(dist1, dist2)
        print("Change of Belif: " + str(tmp[count]["belif"]))
        count += 1
        print("In " + str(layer_number+1) +   " Step Generate: "+str(count))
        entropy_score = ans1[-1]
    return tmp

In [ ]:
# demo 
n_round = 3
i = 9 
n_step = 3
n_shot = 4

step_reasoning_list = []
# print the prompt and generated answer
prompt = n_shot_prompt_generator_GSM8K_reason(n_shot,n_step,GSM8K_test_df["question"][i], GSM8K_train_df_reason, False, n_step, False)
print("-----------------------------------------------------")
# step 1 generation 
step_reasoning = Davinci_openai_stop(prompt,["Step 2"])
step_reasoning_list.append(step_reasoning)
print(step_reasoning['choices'][0]["text"])

ans1 = entropy_check(i,step_reasoning_list , n_shot, n_step)

# new next prompt 
prompt = prompt + step_reasoning["choices"][0]["text"]

# step 2
print("-----------------------------------------------------")
step_reasoning = Davinci_openai_stop(prompt,["Step 3"])
step_reasoning_list.append(step_reasoning)
print(step_reasoning['choices'][0]["text"])

ans2 = entropy_check(i,step_reasoning_list , n_shot, n_step)


# new next prompt
prompt = prompt + step_reasoning["choices"][0]["text"]

# step 3
print("-----------------------------------------------------")
step_reasoning = Davinci_openai_stop(prompt,["Step 4"])
step_reasoning_list.append(step_reasoning)
print(step_reasoning['choices'][0]["text"])

ans3 = entropy_check(i,step_reasoning_list , n_shot, n_step)

In [ ]:
layer_total = 3

n_round = 3
question_index = 9 
n_step = 3
n_shot = 4
tree_structure = {}

record_index_list = []
record_dic_total = {}
divide_list = []
index_dic = 0

for layer_index in range(layer_total):
    if layer_index == 0:
        tmp_list = []
        n_round = 3
        question_index = 9 
        n_step = 3
        n_shot = 4

        log_book = {}
        step_str = "Step{"+str(1) + "}Num"
        print(step_str)
        # print the prompt and generated answer
        log_book[step_str] = 0

        entropy_score = 5 
        # intial prompt 
        count = 0 
        while (entropy_score > 0.6) &  (count < 2):
            step_reasoning_list = []
            prompt = n_shot_prompt_generator_GSM8K_reason(n_shot,n_step,GSM8K_test_df["question"][question_index], GSM8K_train_df_reason, False, n_step, False)
            print("-----------------------------------------------------")
            # step 1 generation 
            step_reasoning = Davinci_openai_stop(prompt,["Step 2"])
            step_reasoning_list.append(step_reasoning)
            print(step_reasoning['choices'][0]["text"])
            
            ans1 = entropy_check(question_index,step_reasoning_list , n_shot, n_step)
            tree_structure[count] = {}
            log_book[count] = {}
            log_book[count]["prompt"] = prompt
            log_book[count]["Status"] = ans1
            log_book[count]["step_reasoning"] = step_reasoning 
            log_book[count]["step_reasoning_list"] = step_reasoning_list
            print('-------------------------------------------')
            for i in step_reasoning_list:
                print(i['choices'][0]["text"])
            print('-------------------------------------------')
            log_book[step_str] += 1
            log_book[count]["layer"] = step_str
            tmp_list.append(count)
            record_dic_total[index_dic] = log_book[count] 
            index_dic += 1
            count += 1
            print("In first Step Generate: "+str(count))
            entropy_score = ans1[-1]
        record_index_list.append(tmp_list)
    else:
        tmp_list = []
        tmp_list2 = []
        step_str = "Step{"+str(layer_index+1) + "}Num"
        print(step_str)
        for i in record_index_list[-1]:
            tmp_book_i = record_dic_total[i]
            ans = iteration(tmp_book_i, 0.001) 
            tmp_num = ans[step_str] 
            for j in range(tmp_num):
                record_dic_total[index_dic] = ans[j]
                tmp_list.append(index_dic)
                index_dic += 1 
            tmp_list2.append(index_dic)
        divide_list.append(tmp_list2)
        record_index_list.append(tmp_list) 
for i,j in enumerate(divide_list):
    divide_list[i] = [k - 1 for k in j]

adjusted_list = []

for i in range(layer_total):
    if i !=0 :
        sub_list = record_index_list[i].copy()
        sub_divide = divide_list[i-1].copy()
        tmp1 = []
        tmp = []
        for j in sub_list:
            if len(sub_divide) != 0:
                if j == sub_divide[0]:
                    sub_divide.remove(sub_divide[0])
                    tmp.append(j)
                    tmp1.append(tmp)
                    tmp = []
                else:
                    tmp.append(j)
            
        adjusted_list.append(tmp1)
    else:
        adjusted_list.append(record_index_list[i])

log_book_copy = log_book.copy()
for i in np.arange(len(adjusted_list)-1, -1, -1):
    if i == 0:
        break
        tmp_list = adjusted_list[i+1]
        step_str = "Step{"+str(i+1) + "}Num"
        for j, sub_list in enumerate(tmp_list):
            setInDict(log_book_copy, [j,"Next"], {})
            setInDict(log_book_copy, [j,"Next",step_str], len(sub_list))
            for k, dic in enumerate(sub_list):
                setInDict(log_book_copy, [j,"Next", k], record_dic_total[dic])
    elif i == 1: 
        step_str = "Step{"+str(i+1) + "}Num"
        for j, position_j in enumerate(record_index_list[i-1]):
            dic = record_dic_total[position_j]
            setInDict(dic, ["Next"], {})
            setInDict(dic, ["Next",step_str], len(sub_list))
            for k, position_k in enumerate(adjusted_list[i][j]):
                setInDict(dic, ["Next", k], record_dic_total[position_k])
            log_book_copy[position_j] = dic         
    else:
        step_str = "Step{"+str(i+1) + "}Num"
        for j, position_j in enumerate(record_index_list[i-1]):
            dic = record_dic_total[position_j]
            setInDict(dic, ["Next"], {})
            setInDict(dic, ["Next",step_str], len(sub_list))
            for k, position_k in enumerate(adjusted_list[i][j]):
                setInDict(dic, ["Next", k], record_dic_total[position_k])
            record_dic_total[position_j] = dic 